In [1]:
library(parallel)
library(stringr)

In [2]:
vcf_dir = '/nfs/lab/projects/pbmc_snATAC/pipeline/snp_array/final_vcfs/'

In [3]:
outdir = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/wasp_peaks/filtered_vcfs'

In [15]:
bed     =  '/nfs/lab/projects/pbmc_snATAC/analysis_v2/peaks/pbmc.sorted.100pad.merged.bed'

### FIlter original vcf files for just  snps in peaks, imputation quality and heterozygous genotype and remove indels

In [5]:
    for (n in 1:22) {

    vcf    = paste0(vcf_dir, "/chr",n ,'.PBMC.topmed.hg19.final.vcf.gz')
      
    outvcf = paste0(outdir, "/chr",n ,'.filt.vcf.gz')
    system(paste('bcftools view', vcf ,
    '-R' , bed, 
    "-i \'INFO/R2[0]>0.7\' | bcftools view --types snps -g het -Oz -o" , outvcf))
 
        system(paste("tabix", outvcf))
    }

In [16]:
#### actually keep the indels
 for (n in 1:22) {

    vcf    = paste0(vcf_dir, "/chr",n ,'.PBMC.topmed.hg19.final.vcf.gz')
      
    outvcf = paste0(outdir, "/chr",n ,'.filt.vcf.gz')
    system(paste('bcftools view', vcf ,
    '-R' , bed, 
    "-i \'INFO/R2[0]>0.7\' | bcftools view -g het -Oz -o" , outvcf))
 
        system(paste("tabix", outvcf))
    }


In [9]:
ids = c(1,3:6, 8:10,12:15)

In [47]:
for (n in ids) {
    from   = paste0('/nfs/lab/projects/pbmc_snATAC/analysis_v2/wasp_peaks/PBMC', n, '/sc_bams')
    newdir = paste0('/nfs/lab/projects/pbmc_snATAC/analysis_v2/wasp_peaks/singlecell/PBMC', n)
    system(paste("mkdir", newdir))
   
    system(paste("mv", from, newdir))
    
    }

In [48]:
dir = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/wasp_peaks/bulk/PBMC3'

In [53]:
tab = data.frame()
for(n in 1:22){
tab1 = read.table(paste0(dir, "/snps/chr", n, ".snps.txt.gz"))
tab1$chr = n    
tab = rbind(tab, tab1)
}

In [54]:
dim(tab)

[1] 500044      4

In [57]:
tab$size = apply(tab, 1, function(x) max(nchar(x[2]), nchar(x[3])))

In [59]:
varlength = table(tab$size)

In [63]:
varlength


     1      2      3      4      5      6      7      8      9     10     11 
468342  17547   4851   2780   2865    924    533    313    309    239    214 
    12     13     14     15     16     17     18     19     20     21     22 
   165    188     92    111     88     88     57     66     55     43     33 
    23     24     25     26     27     28     29     30     31     32     33 
    36     18     16     15      8     17      8      8      1      5      3 
    37     38     40 
     2      1      3 

In [62]:
1-sum((varlength[1])/sum(varlength))

[1] 0.06339842

In [65]:
sum(varlength[2:(length(varlength))])

[1] 31702

In [66]:
tab$SNP = paste(tab$chr, tab$V1,tab$V2,tab$V3, sep=":")

In [67]:
head(tab)

,V1,V2,V3,chr,size,SNP
,<int>,<fct>,<fct>,<int>,<int>,<chr>
1,714277,C,A,1,1,1:714277:C:A
2,751756,T,C,1,1,1:751756:T:C
3,752509,CA,C,1,2,1:752509:CA:C
4,777122,A,T,1,1,1:777122:A:T
5,777232,C,T,1,1,1:777232:C:T
6,779322,A,G,1,1,1:779322:A:G


In [68]:
bed = data.frame(tab$chr, tab$V1-1, tab$V1, tab$SNP)

In [70]:
bed =subset(bed, tab$size>1)

In [71]:
dim(bed)

[1] 31702     4

In [73]:
write.table(bed, '/nfs/lab/projects/pbmc_snATAC/analysis_v2/example_boxplots/indels.bed', sep="\t", quote=F, 
            col.names=F, row.names=F)

In [74]:
examp = read.table('/nfs/lab/projects/pbmc_snATAC/analysis_v2/example_boxplots/snp_list')

In [76]:
ex_bed = data.frame(examp[,1], examp[,2]-10000, examp[,2]+10000, paste(examp[,1], examp[,2], sep=":"))

In [79]:
write.table(ex_bed, '/nfs/lab/projects/pbmc_snATAC/analysis_v2/example_boxplots/snp_list_10kb.bed', sep="\t", quote=F, 
            col.names=F, row.names=F)

In [ ]:
bedtools intersect -a snp_list_10kb.bed -b indels.bed -wa -wo > indels_at_snps.bed